In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH14-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH14-Retriever


# MultiQueryRetriever

1. 개요
    - 기존 방식 : 1개 질문 -> 1개 쿼리 벡터 -> TopK 문서 검색  
    - 쿼리의 세부적인 차이나 임베딩이 데이터 의미를 제대로 포착하지 못할 경우 검색 결과가 달라짐 
    - 사용자 입력 쿼리를 다양한 관점에서 여러 쿼리로 자동으로 생성하여 벡터 검색하여 결과를 통합하여 의미 있는 결과 도출 

2. 주요 특징 
    - 질문 다양화 통한 검색 범위 확장  
        - LLM 기반 쿼리 생성기로부터 다양한 관점의 질문 생성    
    - 검색 결과의 Recall 향상 
        - 단일 질문으로는 찾지 못하는 문서도, 다른 표현의 쿼리 통해 포착 가능   
    - 결과 통합 및 중복 제거 
        - 다양한 쿼리로 찾은 문서를 자동으로 통합 
        - 중복 제거 통해 풍부하면서도 정돈된 검색 결과 제공 


[Reference] https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
openai_embedding = OpenAIEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, openai_embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.invoke(query)

# 검색된 문서의 개수 출력
len(relevant_docs)

4

In [5]:
print(relevant_docs[1].page_content)

가장 강력한 도구로서, Assistant에게 사용자 정의 함수를 지정할 수 있습니다. 이는 Chat Completions API에서의 함수 호출과 매우 유사합니다.


Function calling(함수 호출) 도구를 사용하면 Assistant 에게 사용자 정의 함수 를 설명하여 호출해야 하는 함수를 인자와 함께 지능적으로 반환하도록 할 수 있습니다.


Assistant API는 실행 중에 함수를 호출할 때 실행을 일시 중지하며, 함수 호출 결과를 다시 제공하여 Run 실행을 계속할 수 있습니다. (이는 사용자 피드백을 받아 재게할 수 있는 의미이기도 합니다. 아래 튜토리얼에서 상세히 다룹니다).


## Retriever 정의 

In [6]:
import logging
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

In [7]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm( 
    retriever=db.as_retriever(),
    llm=llm,
)

- 다중 쿼리를 생성하는 중간 과정을 디버깅 하기 위해 정의 

In [8]:
# 쿼리에 대한 로깅 설정
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [9]:
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = multiquery_retriever.invoke(question)

print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API에서 Functions를 사용하는 방법에 대해 설명해 주세요.  ', 'OpenAI Assistant API의 Functions 기능을 활용하는 방법은 무엇인가요?  ', 'OpenAI Assistant API의 Functions 사용에 대한 가이드를 제공해 주실 수 있나요?']


검색된 문서 개수: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


## LangChain 적용 

- User Prompt 정의하고, 정의한 프롬프트와 함께 Chain 생성 

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [11]:
# 프롬프트 템플릿 정의
# 5개의 질문을 생성하도록 프롬프트 구성 
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant. 
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION: 
{question}

#Answer in Korean:
"""
)

# 모델 정의
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Chain 정의 
custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

In [12]:
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

multi_queries = custom_multiquery_chain.invoke(question)
multi_queries

'OpenAI Assistant API의 Functions 기능을 사용하는 방법에 대해 설명해 주세요.  \nOpenAI Assistant API에서 Functions를 활용하는 방법이 궁금합니다.  \nOpenAI Assistant API의 Functions를 어떻게 사용할 수 있는지 알려주세요.  \nOpenAI Assistant API의 Functions 사용법에 대한 정보를 제공해 주세요.  \nOpenAI Assistant API에서 Functions를 사용하는 절차를 설명해 주세요.  '

In [13]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=custom_multiquery_chain, 
    retriever=db.as_retriever(),
)

In [14]:
relevant_docs = multiquery_retriever.invoke(question)

print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API의 Functions 사용법에 대해 설명해 주세요.  ', 'OpenAI Assistant API에서 Functions를 사용하는 방법을 알려주세요.  ', 'OpenAI Assistant API의 Functions 기능에 대한 정보를 제공해 주세요.  ', 'OpenAI Assistant API의 Functions 사용에 관해 자세히 설명해 주실 수 있나요?  ', 'OpenAI Assistant API의 Functions에 대한 사용법을 알고 싶습니다.']


검색된 문서 개수: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


==> 위 실행 내용에 대하여 LangSmith Trace 내역 확인 

-----
** End of Documents **